In [1]:
#Tensorflow version

import tensorflow as tf
print(tf.__version__)

2.9.1


In [2]:
#Importing libraries

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
import matplotlib.pyplot as plt
from keras.applications.efficientnet import EfficientNetB0
from keras.preprocessing import image
#from keras.preprocessing.image import load_img
from keras.models import Sequential
import numpy as np
from tensorflow.keras import regularizers

In [3]:
#Datasets path

# 3 percent with HYP 10 percent (17000)
train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/3 percent with HYP 10 percent - v2 - 17000/Training/"
val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/3 percent with HYP 10 percent - v2 - 17000/Validation/"


In [4]:
from tensorflow.keras import layers

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128, 
                                                                 )
val_data = tf.keras.preprocessing.image_dataset_from_directory(directory=val_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128,
                                                                )

Found 13040 files belonging to 5 classes.


2023-02-17 10:46:25.620090: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-17 10:46:25.648401: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-17 10:46:25.648734: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-17 10:46:25.662268: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 5179 files belonging to 5 classes.


In [5]:
#Normalization of the data

nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)

train_data = train_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

val_data = val_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

In [6]:
# Instantiate base_model with pre-trained weights

#import tensorflow.python.keras.applications.VGG16

base_model= tf.keras.applications.VGG16(weights= 'imagenet', input_shape=(256,256, 3), include_top=False) 

In [7]:
# Freezing the base_model for transfer learning
base_model.trainable= False

In [8]:
# Creating a new_model on the top
inputs= tf.keras.Input(shape= (256, 256, 3), name="input_layer")

In [9]:
x= base_model(inputs,training= False)

In [10]:
# Average pool the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name='Global_Average_Pooling_Layer')(x)

In [11]:
# Output layer
u = tf.keras.layers.Dense(1050, activation="relu")(x)
u = tf.keras.layers.Dense(512, activation="relu")(u)
u = tf.keras.layers.Dense(256, activation="relu")(u)
u = tf.keras.layers.Dense(128, activation="relu")(u)
outputs = tf.keras.layers.Dense(5, activation="softmax",name="output_layer")(u)

In [12]:
# Combine the model
model = tf.keras.Model(inputs, outputs) 

In [13]:
#printing model summary

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 Global_Average_Pooling_Laye  (None, 512)              0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 1050)              538650    
                                                                 
 dense_1 (Dense)             (None, 512)               538112    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                             

In [14]:
# Compile the model

from tensorflow.keras.metrics import SensitivityAtSpecificity,SpecificityAtSensitivity,Recall,Precision

model.compile(loss= tf.losses.categorical_crossentropy, 
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy',SensitivityAtSpecificity(0.5),SpecificityAtSensitivity(0.5),Recall(0.5),Precision(0.5)])

In [15]:
#callback

class myCallback(tf.keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.99): 
          print("\nReached 99% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [16]:
# Fit the model
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("ECG_Spectrogram_Model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
history = model.fit(train_data, steps_per_epoch=len(train_data), epochs=50, validation_data=val_data, validation_steps=len(val_data),callbacks = [callbacks,checkpoint])

Epoch 1/50


2023-02-17 10:46:34.016395: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-17 10:46:34.269470: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


102/102 [==============================] - ETA: 0s - loss: 1.2901 - accuracy: 0.4711 - sensitivity_at_specificity: 0.8518 - specificity_at_sensitivity: 0.8552 - recall: 0.1989 - precision: 0.6296
Epoch 1: val_accuracy improved from -inf to 0.55204, saving model to ECG_Spectrogram_Model.h5
102/102 [==============================] - 105s 920ms/step - loss: 1.2901 - accuracy: 0.4711 - sensitivity_at_specificity: 0.8518 - specificity_at_sensitivity: 0.8552 - recall: 0.1989 - precision: 0.6296 - val_loss: 1.1346 - val_accuracy: 0.5520 - val_sensitivity_at_specificity: 0.9104 - val_specificity_at_sensitivity: 0.9129 - val_recall: 0.4072 - val_precision: 0.6533
Epoch 2/50
102/102 [==============================] - ETA: 0s - loss: 1.0632 - accuracy: 0.5849 - sensitivity_at_specificity: 0.9311 - specificity_at_sensitivity: 0.9338 - recall: 0.4215 - precision: 0.7135
Epoch 2: val_accuracy improved from 0.55204 to 0.60533, saving model to ECG_Spectrogram_Model.h5
102/102 [========================

Epoch 14/50
102/102 [==============================] - ETA: 0s - loss: 0.7587 - accuracy: 0.7145 - sensitivity_at_specificity: 0.9823 - specificity_at_sensitivity: 0.9794 - recall: 0.6313 - precision: 0.7810
Epoch 14: val_accuracy improved from 0.68527 to 0.70844, saving model to ECG_Spectrogram_Model.h5
102/102 [==============================] - 83s 813ms/step - loss: 0.7587 - accuracy: 0.7145 - sensitivity_at_specificity: 0.9823 - specificity_at_sensitivity: 0.9794 - recall: 0.6313 - precision: 0.7810 - val_loss: 0.7782 - val_accuracy: 0.7084 - val_sensitivity_at_specificity: 0.9759 - val_specificity_at_sensitivity: 0.9779 - val_recall: 0.6271 - val_precision: 0.7810
Epoch 15/50
102/102 [==============================] - ETA: 0s - loss: 0.7424 - accuracy: 0.7211 - sensitivity_at_specificity: 0.9812 - specificity_at_sensitivity: 0.9816 - recall: 0.6397 - precision: 0.7853
Epoch 15: val_accuracy did not improve from 0.70844
102/102 [==============================] - 83s 810ms/step - lo

Epoch 27/50
102/102 [==============================] - ETA: 0s - loss: 0.6518 - accuracy: 0.7531 - sensitivity_at_specificity: 0.9896 - specificity_at_sensitivity: 0.9877 - recall: 0.6899 - precision: 0.8078
Epoch 27: val_accuracy did not improve from 0.72253
102/102 [==============================] - 83s 810ms/step - loss: 0.6518 - accuracy: 0.7531 - sensitivity_at_specificity: 0.9896 - specificity_at_sensitivity: 0.9877 - recall: 0.6899 - precision: 0.8078 - val_loss: 0.7911 - val_accuracy: 0.7138 - val_sensitivity_at_specificity: 0.9697 - val_specificity_at_sensitivity: 0.9782 - val_recall: 0.6571 - val_precision: 0.7680
Epoch 28/50
102/102 [==============================] - ETA: 0s - loss: 0.6310 - accuracy: 0.7617 - sensitivity_at_specificity: 0.9886 - specificity_at_sensitivity: 0.9889 - recall: 0.7045 - precision: 0.8144
Epoch 28: val_accuracy did not improve from 0.72253
102/102 [==============================] - 83s 810ms/step - loss: 0.6310 - accuracy: 0.7617 - sensitivity_at

Epoch 40/50
102/102 [==============================] - ETA: 0s - loss: 0.5653 - accuracy: 0.7889 - sensitivity_at_specificity: 0.9923 - specificity_at_sensitivity: 0.9927 - recall: 0.7387 - precision: 0.8317
Epoch 40: val_accuracy did not improve from 0.72253
102/102 [==============================] - 83s 809ms/step - loss: 0.5653 - accuracy: 0.7889 - sensitivity_at_specificity: 0.9923 - specificity_at_sensitivity: 0.9927 - recall: 0.7387 - precision: 0.8317 - val_loss: 0.8512 - val_accuracy: 0.7057 - val_sensitivity_at_specificity: 0.9637 - val_specificity_at_sensitivity: 0.9780 - val_recall: 0.6617 - val_precision: 0.7492
Epoch 41/50
102/102 [==============================] - ETA: 0s - loss: 0.5338 - accuracy: 0.8000 - sensitivity_at_specificity: 0.9961 - specificity_at_sensitivity: 0.9935 - recall: 0.7554 - precision: 0.8415
Epoch 41: val_accuracy did not improve from 0.72253
102/102 [==============================] - 83s 811ms/step - loss: 0.5338 - accuracy: 0.8000 - sensitivity_at

In [17]:
#Acquiring Performance metrics

Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity']
Validation_Recall=history.history['val_recall']
Validation_Precision=history.history['val_precision']
Validation_Loss=history.history['val_loss']

print("Training Accuracy: ",max(Training_Accuracy))
print("Validation Accuracy: ",max(Validation_Accuracy))
print("Validation Specificity: ",max(Validation_Specificity))
print("Validation Sensitivity: ",max(Validation_Sensitivity))
print("Validation Recall: ",max(Validation_Recall))
print("Validation Precision: ",max(Validation_Precision))
print("Validation Loss: ",min(Validation_Loss))

Training Accuracy:  0.8212423324584961
Validation Accuracy:  0.7225332856178284
Validation Specificity:  0.9802085161209106
Validation Sensitivity:  0.9770225882530212
Validation Recall:  0.6725236773490906
Validation Precision:  0.7888431549072266
Validation Loss:  0.7752128839492798
